To switch to dbt profile with the `arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo` role that I created

In [ ]:
nano ~/.aws/config

# In nano add this profile
[profile dbt-glue]
role_arn = arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo
source_profile = Joshua_Harris_ETL
region = ap-southeast-2

# switch profile for the terminal session
export AWS_PROFILE=dbt-glue

# check your cureent profile
aws sts get-caller-identity

lets make a csv file with example data

In [29]:
# pip install pandas numpy

In [30]:
from pandas import DataFrame
from numpy import nan

import numpy as np

def permute_dataframe(df, random_state=None):
    """
    Returns a new DataFrame with the rows randomly permuted.
    Each call produces a different order unless random_state is set.
    """
    return df.sample(frac=1, random_state=random_state).reset_index(drop=True)


dirty_df = DataFrame({
    'patient_id': ["A001", "A002", "A003", "A004", "A005", ""], 
    'sex': ["M", "F", "0", "1", "Male", "fem"],
    'medication': [None, '', None, nan, None, 'statin'],
    'age': [nan, 32, 214, 21, 0, ""]
    })


# Example usage of permute_dataframe:

import random
permuted_df = permute_dataframe(dirty_df, random_state=random.randint(0, 1_000_000))


with open('../data/patient_info.csv', 'w') as f:
    permuted_df.to_csv(f, index=False)

lets upload this file to s3

In [31]:
%%bash
DATETIME=$(date +%Y%m%d%H%M%S)
DATA_NAME="patient_info"
aws s3 cp "../data/patient_info.csv" "s3://acdc-dbt-test-raw/data/${DATA_NAME}/${DATETIME}/${DATA_NAME}.csv" --profile "default"

upload: ../data/patient_info.csv to s3://acdc-dbt-test-raw/data/patient_info/20250725173021/patient_info.csv


lets make some other data types

In [32]:
from pandas import DataFrame
from numpy import nan

dirty_df = DataFrame({
    'sample_id': ["SAM0001", "SAM0002", "SAM0003", "SAM0004", "SAM0005", ""],
    'patient_id': ["A001", "A002", "A003", "A004", "A005", ""], 
    'storage_medium': ["nitrogen", "air", "nitrogen", "air", "nitrogen", "air"],
    'storage_date': ["2022-01-01", "2022-01-02", "2022-01-03", "2022-01-04", "2022-01-05", "2022-01-06"],
    'volume_ul': [0, 486, 4, 4000, 5000, 6000]
    })

permuted_df = permute_dataframe(dirty_df, random_state=random.randint(0, 1_000_000))

with open('../data/sample.csv', 'w') as f:
    permuted_df.to_csv(f, index=False)

In [33]:
%%bash
DATETIME=$(date +%Y%m%d%H%M%S)
DATA_NAME="sample"
aws s3 cp "../data/sample.csv" "s3://acdc-dbt-test-raw/data/${DATA_NAME}/${DATETIME}/${DATA_NAME}.csv" --profile "default"

upload: ../data/sample.csv to s3://acdc-dbt-test-raw/data/sample/20250725173021/sample.csv


# Trying to run dbt

### IAM roles:

The principle role: `arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo` as the following policies attached:
-  `arn:aws:iam::026090528544:policy/acdc-dbt-glue-test`
-  `arn:aws:iam::026090528544:policy/GlueInteractiveSessionClientPolicy-dbt`
        - This role Grants permission to create and manage AWS Glue interactive sessions and pass the designated execution role required to run dbt workloads in AWS Glue.

Once glue has been created, then `AwsGlueSessionUserRestrictedServiceRole-dbtGlueExecutor` role which is defined in `profiles.yml` will then be used by the glue engine to run spark statements. This role contains the following policies:
- `arn:aws:iam::aws:policy/service-role/AwsGlueSessionUserRestrictedServiceRole`
- `arn:aws:iam::026090528544:policy/acdc-dbt-glue-test`

***

### 🔐 IAM Roles for `dbt` + AWS Glue Interactive Sessions

Running `dbt` on AWS Glue using interactive sessions requires **two IAM roles**, each with a distinct purpose:

---

#### 1. 🧑‍💻 Client Principal Role

**Role ARN:**  
`arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo`

This role is **assumed by you** (or your CI/CD runner) when you execute `dbt` commands from the CLI. It represents the identity that **creates and manages** interactive Glue sessions.

**Attached Policies:**
- `arn:aws:iam::026090528544:policy/acdc-dbt-glue-test`
- `arn:aws:iam::026090528544:policy/GlueInteractiveSessionClientPolicy-dbt`

**What this role allows:**
- Creating Glue sessions:  
  `glue:CreateSession`, `glue:GetSession`, `glue:CancelStatement`, `glue:RunStatement`
- Passing execution roles to the Glue service:  
  `iam:PassRole`

> 💡 *Think of this as the **"session initiator"** — the identity that opens the workspace and tells Glue what role to use inside.*

---

#### 2. ⚙️ Executor (Runtime) Role

**Role ARN (defined in `profiles.yml`):**  
`arn:aws:iam::026090528544:role/AwsGlueSessionUserRestrictedServiceRole-dbtGlueExecutor`

This role is **assumed by Glue** after a session is created. It is used by the Spark engine to **execute your dbt workload inside Glue**.

**Attached Policies:**
- `arn:aws:iam::aws:policy/service-role/AwsGlueSessionUserRestrictedServiceRole`
- `arn:aws:iam::026090528544:policy/acdc-dbt-glue-test`

**What this role allows:**
- Accessing AWS Glue Data Catalog:  
  `glue:GetDatabase`, `glue:GetTable`, `glue:GetPartition`, etc.
- Accessing and writing to S3:  
  `s3:GetObject`, `s3:PutObject`, `s3:ListBucket`

> 💡 *Think of this as the **"Glue worker"** — the identity that Spark uses to run your SQL models and materialize results.*

---

### 🧠 Summary Table

| Role Name                                               | Used By       | Purpose                      | Key Permissions                             |
|----------------------------------------------------------|----------------|-------------------------------|----------------------------------------------|
| `GlueInteractiveSessionClientRole-dbt-CLI-demo`          | dbt CLI / You | Create and manage Glue sessions | `glue:*Session*`, `glue:RunStatement`, `iam:PassRole` |
| `AwsGlueSessionUserRestrictedServiceRole-dbtGlueExecutor` | AWS Glue       | Run Spark code inside session | `glue:Get*`, `s3:*`, Spark runtime           |

In [34]:
# Checking dbt status to see if it connected and ready
# !dbt debug # Run in terminal

Should look like:

```
dbt debug                                                                                                                                                                                                                                                                                                            (main✱) 
04:33:16  Running with dbt=1.9.8
04:33:16  dbt version: 1.9.8
04:33:16  python version: 3.9.5
04:33:16  python path: /Users/harrijh/.pyenv/versions/3.9.5/bin/python3.9
04:33:16  os info: macOS-15.5-arm64-arm-64bit
04:33:16  Using profiles dir at /Users/harrijh/projects/acdc-datapipeline-demo/dbt_demo_acdc
04:33:16  Using profiles.yml file at /Users/harrijh/projects/acdc-datapipeline-demo/dbt_demo_acdc/profiles.yml
04:33:16  Using dbt_project.yml file at /Users/harrijh/projects/acdc-datapipeline-demo/dbt_demo_acdc/dbt_project.yml
04:33:16  adapter type: glue
04:33:16  adapter version: 1.9.4
04:33:16  Configuration:
04:33:16    profiles.yml file [OK found and valid]
04:33:16    dbt_project.yml file [OK found and valid]
04:33:16  Required dependencies:
04:33:16   - git [OK found]

04:33:16  Connection:
04:33:16    role_arn: arn:aws:iam::026090528544:role/AwsGlueSessionUserRestrictedServiceRole-dbtGlueExecutor
04:33:16    region: ap-southeast-2
04:33:16    workers: 2
04:33:16    worker_type: G.1X
04:33:16    session_provisioning_timeout_in_seconds: 120
04:33:16    schema: glue-db-acdc-dbt-test-clean
04:33:16    location: s3://acdc-dbt-test-clean/dbt/
04:33:16    extra_jars: None
04:33:16    idle_timeout: 10
04:33:16    query_timeout_in_minutes: 300
04:33:16    glue_version: 5.0
04:33:16    security_configuration: None
04:33:16    connections: None
04:33:16    conf: None
04:33:16    extra_py_files: None
04:33:16    delta_athena_prefix: None
04:33:16    tags: None
04:33:16    seed_format: parquet
04:33:16    seed_mode: overwrite
04:33:16    default_arguments: None
04:33:16    iceberg_glue_commit_lock_table: myGlueLockTable
04:33:16    use_interactive_session_role_for_api_calls: False
04:33:16    lf_tags: None
04:33:16    glue_session_id: None
04:33:16    glue_session_reuse: True
04:33:16    datalake_formats: None
04:33:16    enable_session_per_model: False
04:33:16    use_arrow: False
04:33:16    enable_spark_seed_casting: False
04:33:16  Registered adapter: glue=1.9.4
04:33:50    Connection test: [OK connection ok]

04:33:50  All checks passed!
```

running my new model `dbt_demo_acdc/models/staging/stg_patient_info.sql`

In [35]:
# dbt run --select stg_patient_info

# Here is what DBT is doing with versioning

Based on the advanced, versioned project structure we've designed, here is a detailed breakdown of what will happen when you run the command `dbt run --select stg_patient_info`.

This command now triggers a more sophisticated process because you've implemented dbt model versioning and a dynamic, immutable S3 release path.

### The Step-by-Step Execution Flow

When you execute `dbt run --select stg_patient_info`, dbt will perform the following steps:

1.  **Model Resolution**: The command `stg_patient_info` refers to the *conceptual model*. dbt will look at your `models/staging/patient_info/_patient_info__models.yml` file to determine which version of this model to run. It sees the line `latest_version: 1`, so it knows to execute the logic defined for `v1`.

2.  **Code Execution**: dbt finds the `v1` model, which points to the file `stg_patient_info_v1.sql`. It will execute the SQL transformation logic contained within that specific file.

3.  **Dynamic Path Construction**: dbt then looks at your `profiles.yml` to determine where to save the output. It finds the `location` parameter:
    `location: "s3://acdc-dbt-test-clean/releases/{{ var('release_version', 'default') }}/"`
    It then checks your `dbt_project.yml` for the value of the `release_version` variable, which is set to `"v1.0"`.

4.  **Final Output Path**: dbt constructs the final, immutable S3 path for this run, which will be:
    `s3://acdc-dbt-test-clean/releases/v1.0/stg_patient_info/`

### The Two Possible Outcomes

What happens next depends entirely on whether this is the first time you are creating the `v1.0` release.

#### Scenario A: This is the first time you are creating the "v1.0" release

If the directory `s3://acdc-dbt-test-clean/releases/v1.0/` does not yet exist, the command will **succeed**.

*   The AWS Glue job will run.
*   The `stg_patient_info_v1.sql` logic will transform the data.
*   The clean data will be written as new Parquet files to the new `.../releases/v1.0/stg_patient_info/` folder.
*   A table definition for `stg_patient_info` will be created in your `glue_db_acdc_dbt_test_clean` database, pointing to this new, versioned S3 location.

#### Scenario B: You have already created the "v1.0" release before

If you have already run this command successfully once, the directory `.../releases/v1.0/stg_patient_info/` will already exist and contain data. In this case, your `dbt run` command will **fail** with the same error you encountered previously[1][2][3]:

`AnalysisException: CREATE-TABLE-AS-SELECT cannot create table with location to a non-empty directory...`

This is the expected safety feature preventing you from accidentally corrupting a data release.

### How to Proceed in Scenario B

If you encounter the "non-empty directory" error for your `v1.0` release and you intentionally want to rebuild it from scratch, you must use the `--full-refresh` flag[4].

```bash
dbt run --select stg_patient_info --full-refresh
```

This command tells dbt to first delete the existing data files in `s3://.../releases/v1.0/stg_patient_info/` before running the `CREATE TABLE` command, ensuring the run will succeed.

By implementing this versioned structure, you have established a robust pattern for managing your data releases. Each new version of your data will live in its own immutable S3 folder, creating a clear, auditable history that is perfect for reproducible research.

[1] https://community.databricks.com/t5/data-engineering/error-the-associated-location-is-not-empty-but-it-s-not-a-delta/td-p/7428
[2] https://github.com/databricks/dbt-databricks/issues/771
[3] https://github.com/trinodb/trino/issues/20749
[4] https://hevodata.com/data-transformation/dbt-full-refresh/
[5] https://stackoverflow.com/questions/77472158/iceberg-filesystem-error-cannot-create-a-table-on-a-non-empty-location
[6] https://discourse.getdbt.com/t/create-empty-table-through-dbt-seed/11667
[7] https://docs.getdbt.com/reference/resource-properties/versions
[8] https://www.linkedin.com/pulse/mastering-breaking-changes-safely-removing-columns-dbt-joshua-odeyemi-5xskf
[9] https://www.getgalaxy.io/learn/glossary/dbt-snapshot
[10] https://www.datafold.com/blog/catching-unintended-changes-to-immutable-data
[11] https://community.databricks.com/t5/get-started-discussions/bug-cannot-create-table-the-associated-location-is-not-empty-and/td-p/37335
[12] https://popsql.com/learn-dbt/dbt-full-refresh
[13] https://altisconsulting.com/au/insights/leveraging-dbt-model-versions-to-enhance-trust-and-reliability/
[14] https://handbook.gitlab.com/handbook/enterprise-data/how-we-work/dbt-change-workflow/
[15] https://discourse.getdbt.com/t/model-version-of-a-snapshot/17677
[16] https://www.getdbt.com/blog/introducing-release-tracks-for-dbt-version-upgrades
[17] https://www.datacamp.com/tutorial/databricks-create-table
[18] https://docs.getdbt.com/docs/build/incremental-models
[19] https://docs.getdbt.com/docs/mesh/govern/model-versions
[20] https://discourse.getdbt.com/t/dbt-support-for-schema-versioned-database-objects/6131